## Import libraries

In [53]:
from pytwitter import Api
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer


In [3]:
api = Api(bearer_token="AAAAAAAAAAAAAAAAAAAAAMhqlAEAAAAA4Pqzn354Z5nlkP5lKaW98vzlVlA%3D7GIA03xacVKdFYTFg7qmgvWTZThpa2FFd4SNPUqP7uPK7Xjue5")

## Calculates the user's influence and reputation

## Calculates Graph Metrics

## Calculates Text Metrics

In [49]:
def analyze_social_capital(tweets):
  # create an empty directed graph
  graph = nx.DiGraph()
  
  # iterate over the tweets and add edges to the graph
  # representing the relationships between users
  for tweet in tweets:
    user = tweet.author_id
    mentions = tweet.entities.mentions if tweet.entities is None else []
    hashtags = tweet.entities.hashtags if tweet.entities is None else []
    retweets = []
    replies = []

    if(tweet.referenced_tweets is not None):
        for referencedTweet in tweet.referenced_tweets:
          if referencedTweet.type == "replied_to":
            replies.append(referencedTweet.id)
          elif referencedTweet.type == "retweeted":           
            retweets.append(referencedTweet.id)
    
    #likes = tweet.public_metrics.like_count
    
    # add an edge for each mention
    for mention in mentions:
      graph.add_edge(user, mention)

    for hashtag in hashtags:
      graph.add_edge(user, hashtag)
      
    # add an edge for each retweet
    for retweet in retweets:
      graph.add_edge(user, retweet)

    for reply in replies:
      graph.add_edge(user, reply)
      
    # add an edge for each like
    #for like in likes:
    #  graph.add_edge(user, like)

  social_capital = {}
      
  # compute various measures of social capital for each node in the graph
  for node in graph.nodes():
    # compute the in-degree centrality
    in_degree = graph.in_degree(node)
    
    # compute the out-degree centrality
    out_degree = graph.out_degree(node)
    
    # compute the betweenness centrality
    betweenness = nx.betweenness_centrality(graph)[node]
    
    # compute the PageRank
    pagerank = nx.pagerank(graph)[node]
    
    # store the measures of social capital for the current node
    social_capital[node] = {
      'in_degree': in_degree,
      'out_degree': out_degree,
      'betweenness': betweenness,
      'pagerank': pagerank
    }

  nx.draw(graph)
    
  return social_capital

In [ ]:
public_tweets = api.search_tweets(query="stf lang:pt has:hashtags is:retweet", expansions=["referenced_tweets.id.author_id","in_reply_to_user_id","attachments.media_keys","author_id","entities.mentions.username"],
user_fields=["created_at","entities","id","location","name","pinned_tweet_id","profile_image_url","protected","public_metrics","url","username","verified"],
tweet_fields=["attachments","author_id","context_annotations","created_at","entities","geo","in_reply_to_user_id","lang","public_metrics","reply_settings","source"], max_results=100)

api.get_followers()

public_tweets.includes.tweets

#analyze_social_capital(public_tweets.data)

In [ ]:

def social_capital_impact(tweet, social_connections, interactions, texts, resources, user_popularity, user_influence):
  # extract the user, mentions, and hashtags from the tweet
  user = tweet['user']
  mentions = tweet['mentions']
  hashtags = tweet['hashtags']
  
  # compute the number of connections the user has in the social network
  user_connections = np.sum(social_connections[user])
  
  # compute the number of connections between the user and the mentions
  mention_connections = 0
  for mention in mentions:
    mention_connections += social_connections[user][mention]
  
  # compute the number of connections between the user and the hashtags
  hashtag_connections = 0
  for hashtag in hashtags:
    hashtag_connections += social_connections[user][hashtag]
  
  # compute the sum of the resources associated with the tweet
  resource_sum = 0
  for resource in resources:
    if resource in tweet['resources']:
      resource_sum += resources[resource]
  
  # compute the number of interactions for the tweet
  interaction_sum = interactions[tweet['tweet_id']]
  
  # compute the text similarity between the tweet and the other texts
  vectorizer = TfidfVectorizer()
  tweet_text = [tweet['text']]
  tweet_vector = vectorizer.fit_transform(tweet_text)
  other_vectors = vectorizer.transform(texts)
  similarity = cosine_similarity(tweet_vector, other_vectors)
  
  # compute the user popularity and influence factors
  popularity_factor = user_popularity[user]
  influence_factor = user_influence[user]
  
  # compute the social capital impact as the sum of mention connections, hashtag connections, resource sum, interaction sum,
  # text similarity, popularity factor, and influence factor divided by the number of user connections
  impact = (mention_connections + hashtag_connections + resource_sum + interaction_sum + similarity + popularity_factor + influence_factor) / user_connections
  
  return impact
